In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
import glob
import re
import os
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import r2_score

# Log kirliliğini önle
optuna.logging.set_verbosity(optuna.logging.WARNING)

# =============================================================================
# 1. VERİ YÜKLEME VE SİLİKON ANALİZİ (ÖZEL DİKİM)
# =============================================================================
def load_and_analyze_data_custom():
    print("📂 Dosyalar taranıyor ve SENİN SİSTEMİNE ÖZEL analiz ediliyor...")
    
    # Senin dosya isimlerinden ayarları çıkaran kurallar
    file_rules = [
        {"pattern": "Fans ve apu down", "PPT": 142, "CO": 0,   "Boost": 0},
        {"pattern": "-30 CCD",          "PPT": 95,  "CO": -30, "Boost": 200},
        {"pattern": "65W-85W",          "PPT": 65,  "CO": -25, "Boost": 0},
        {"pattern": "95W-115W",         "PPT": 95,  "CO": -20, "Boost": 0}
    ]
    
    csv_files = glob.glob("*.csv") + glob.glob("*.CSV")
    df_list = []
    
    # SENİN PAYLAŞTIĞIN LİSTEYE GÖRE HASSAS EŞLEŞTİRME
    # Kodun temizlik fonksiyonu parantezleri sildiği için hedef isimler şöyle olacak:
    target_cols = {
        'Temp':  'CPU Tctl/Tdie °C',             # Orijinal: CPU (Tctl/Tdie) [°C]
        'Clock': 'Çekirdek Etkili Saatler avg MHz', # Orijinal: Çekirdek Etkili Saatler (avg) [MHz]
        'Power': 'CPU Tüm Güç Tüketimi W',       # Orijinal: CPU Tüm Güç Tüketimi [W]
        'Fan':   'CPU RPM'                       # Orijinal: CPU [RPM]
    }
    
    for f in csv_files:
        rule = next((r for r in file_rules if r["pattern"] in f), None)
        if not rule: continue
        
        try:
            # Veriyi oku
            temp = pd.read_csv(f, low_memory=False, on_bad_lines='skip')
            
            # Sütun İsim Temizliği (Senin listenle uyumlu hale getirme)
            # [], (), boşlukları temizle
            temp.columns = [c.strip().replace('[', '').replace(']', '').replace('(', '').replace(')', '') for c in temp.columns]
            
            # Sütunları Kontrol Et ve Yeniden Adlandır
            rename_map = {}
            for key, target_name in target_cols.items():
                if target_name in temp.columns:
                    rename_map[target_name] = key
            
            # Eğer ana sütunlar eksikse o dosyayı atla
            if len(rename_map) < 3: 
                print(f"  ⚠️ {f} dosyasında kritik sütunlar eksik, atlanıyor.")
                continue
                
            temp = temp.rename(columns=rename_map)
            
            # Feature Engineering
            temp['Set_PPT'] = rule['PPT']
            temp['Set_CO'] = rule['CO']
            temp['Set_Boost'] = rule['Boost']
            # Lag Features
            temp['Temp_Lag1'] = temp['Temp'].shift(1).bfill()
            temp['Clock_Lag1'] = temp['Clock'].shift(1).bfill()
            temp['Aggressiveness'] = temp['Set_PPT'] * abs(temp['Set_CO'])
            
            df_list.append(temp)
            print(f"  -> {f} başarıyla işlendi.")
            
        except Exception as e:
            print(f"  ❌ HATA: {f} okunamadı. Sebebi: {e}")

    if not df_list: raise ValueError("Hiçbir dosya yüklenemedi!")
    full_df = pd.concat(df_list, ignore_index=True)
    
    # ---------------------------------------------------------
    # 1.2. ÇEKİRDEK KALİTE TESPİTİ (T0 ODAKLI)
    # ---------------------------------------------------------
    print("\n🔬 SİLİKON KALİTE ANALİZİ (P95 Yöntemi)...")
    
    core_cols = []
    # Senin listendeki format: "Core 0 T0 Etkili Saat [MHz]" -> Temiz hali: "Core 0 T0 Etkili Saat MHz"
    for i in range(16): # 6 çekirdek var ama 16'ya kadar tarayalım
        col_name = f"Core {i} T0 Etkili Saat MHz"
        if col_name in full_df.columns:
            core_cols.append({"Core": i, "Col": col_name})
    
    core_ranking = []
    analysis_report = []

    if core_cols:
        # FİLTRE: İşlemci gerçekten yük altında mı? (Power > 50W)
        # Ve Çekirdek uyuyor mu? (Clock > 3000 MHz)
        for c in core_cols:
            active_data = full_df[ (full_df['Power'] > 50) & (full_df[c['Col']] > 3000) ]
            
            if len(active_data) < 50:
                score = -1 
                p95 = 0
            else:
                # P95: Çekirdeğin %95 oranında erişebildiği kararlı tepe hız
                p95 = active_data[c['Col']].quantile(0.95)
                # Stabilite cezası (Çok dalgalanıyorsa puan kır)
                stability_penalty = active_data[c['Col']].std() * 0.5
                score = p95 - stability_penalty
            
            analysis_report.append({"Core": c['Core'], "Score": score, "P95": p95, "Count": len(active_data)})
            
        # Puanına göre sırala
        analysis_report.sort(key=lambda x: x['Score'], reverse=True)
        
        print("\n📊 ÇEKİRDEK PERFORMANS TABLOSU:")
        print(f"{'Çekirdek':<10} | {'Max Hız (P95)':<15} | {'Veri Sayısı':<12} | {'Sınıf'}")
        print("-" * 60)
        
        for i, r in enumerate(analysis_report):
            if r['Score'] == -1:
                rank = "Unknown"; offset = 300; safe_co = -15
            elif i < 2:
                rank = "🏆 GOLDEN"; offset = 380; safe_co = -30
            elif i < len(analysis_report) - 2:
                rank = "🥈 SILVER"; offset = 340; safe_co = -25
            else:
                rank = "🥉 BRONZE"; offset = 300; safe_co = -20
                
            print(f"Core {r['Core']:<5} | {r['P95']:.1f} MHz     | {r['Count']:<12} | {rank}")
            
            core_ranking.append({
                "Core": f"Core {r['Core']}",
                "Rank": rank,
                "Offset": offset,
                "Safe_CO": safe_co,
                "Score": r['Score']
            })
    else:
        print("⚠️ UYARI: 'Core X T0 Etkili Saat' sütunları bulunamadı. Lütfen CSV formatını kontrol et.")
        # Fallback
        for i in range(6): core_ranking.append({"Core": f"Core {i}", "Rank": "Unknown", "Offset": 300, "Safe_CO": -15})
        
    X = full_df[['Set_PPT', 'Set_CO', 'Set_Boost', 'Fan', 'Temp_Lag1', 'Clock_Lag1', 'Aggressiveness']]
    y_temp, y_clock = full_df['Temp'], full_df['Clock']
    
    return X, y_temp, y_clock, core_ranking

# =============================================================================
# 2. XGBOOST OPTİMİZASYONU (SABİT 75 TRIAL - AĞIR EĞİTİM)
# =============================================================================
def optimize_and_train(X, y, name):
    print(f"\n🔍 {name} Modeli Optimize Ediliyor (Bu işlem işlemcini biraz yoracak)...")
    
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 800, 2000),
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'n_jobs': -1, 'random_state': 42
        }
        model = xgb.XGBRegressor(**params)
        # 5-Fold CV ile sağlam doğrulama
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
        return -scores.mean()

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=75) # 75 Deneme
    
    print(f"  ✅ {name} En İyi RMSE: {study.best_value:.4f}")
    
    # Final Modeli Eğit
    final_model = xgb.XGBRegressor(**study.best_params)
    final_model.fit(X, y)
    return final_model

# =============================================================================
# 3. SENARYO ÇÖZÜCÜ (TERMAL HEDEF ODAKLI)
# =============================================================================
def solve_scenarios(model_temp, model_clock, core_ranking):
    scenarios = [
        {"Name": "❄️ SESSİZ (75°C)", "Limit": 75},
        {"Name": "⚖️ DENGELİ (85°C)", "Limit": 85},
        {"Name": "🚀 TURBO (95°C)",   "Limit": 95}
    ]
    results = []
    
    for c in core_ranking:
        if c['Score'] == -1: continue 
        
        row = {"Çekirdek": c['Core'], "Kalite": c['Rank']}
        
        # Simülasyon: Kış Modu (120W), Boost +200, Fan 1500 RPM
        # Model tahmini + Çekirdek Kalitesi (Offset)
        for s in scenarios:
            input_data = pd.DataFrame([{
                'Set_PPT': 120, 'Set_CO': c['Safe_CO'], 'Set_Boost': 200, 'Fan': 1500,
                'Temp_Lag1': s['Limit']-5, 'Clock_Lag1': 5000, 'Aggressiveness': 120*abs(c['Safe_CO'])
            }])
            
            # Model ne diyor?
            base_mhz = model_clock.predict(input_data)[0]
            # Kaliteye göre düzeltme
            final_mhz = min(base_mhz + c['Offset'], 5550)
            
            row[s['Name']] = f"CO {c['Safe_CO']} ➡ {final_mhz:.0f} MHz"
            
        results.append(row)
    
    return pd.DataFrame(results)

# =============================================================================
# MAIN
# =============================================================================
if __name__ == "__main__":
    try:
        X, y_temp, y_clock, core_ranking = load_and_analyze_data_custom()
    except Exception as e:
        print(f"HATA: {e}")
        exit()
    
    # Train/Test
    X_train, X_test, y_t_train, y_t_test, y_c_train, y_c_test = train_test_split(X, y_temp, y_clock, test_size=0.2, random_state=42)
    
    # Modelleme
    m_temp = optimize_and_train(X_train, y_t_train, "Sıcaklık")
    m_clock = optimize_and_train(X_train, y_c_train, "Hız")
    
    # R2 Kontrolü
    r2_t = r2_score(y_t_test, m_temp.predict(X_test))
    r2_c = r2_score(y_c_test, m_clock.predict(X_test))
    
    print("\n" + "="*80)
    print(f"🎓 MODEL KARNESİ: Sıcaklık R²= %{r2_t*100:.1f} | Hız R²= %{r2_c*100:.1f}")
    print("="*80)
    
    df_final = solve_scenarios(m_temp, m_clock, core_ranking)
    print("\n🎯 NİHAİ PBO REÇETESİ (Senin Sistemine Özel):")
    print(df_final.to_markdown(index=False))

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import glob

# Önceki hücrelerden gelen modelleri ve verileri kullanıyoruz
# Eğer değişkenler hafızada yoksa lütfen en baştaki yükleme kodunu tekrar çalıştır.

def calibrate_and_simulate_5500(model_temp, model_clock, core_ranking, full_df):
    print("\n🔧 SİSTEM KALİBRASYONU BAŞLATILIYOR (Reality Check)...")
    print("="*80)
    
    TARGET = 5500
    results = []
    
    # 1. GERÇEK VERİDEN KALİBRASYON (Calibration from Ground Truth)
    # Her çekirdek için verisetinde görülmüş EN YÜKSEK hızı buluyoruz.
    
    calibrated_cores = []
    
    print(f"{'Çekirdek':<10} | {'Veri Seti Max':<15} | {'Eski Offset':<12} | {'Yeni (Real) Offset'}")
    print("-" * 65)
    
    for c in core_ranking:
        core_id = int(c['Core'].split()[-1]) # "Core 0" -> 0
        col_name = f"Core {core_id} T0 Etkili Saat MHz"
        
        if col_name in full_df.columns:
            # Veri setindeki mutlak zirve noktası
            real_max_seen = full_df[col_name].max()
            
            # Eğer veri setinde 5449 gibi yüksek bir değer varsa onu baz al
            # Yoksa kullanıcının beyanını (5449) manuel referans alabiliriz ama
            # şimdilik veriden gidiyoruz.
            
            # Modelin bu Max anındaki şartlara (tahmini) tepkisi
            # Basitlik için standart bir yük senaryosunu baz alıp farkı bulacağız
            dummy_input = pd.DataFrame([{
                'Set_PPT': 95, 'Set_CO': -20, 'Set_Boost': 200, 'Fan': 1500,
                'Temp_Lag1': 75, 'Clock_Lag1': 5000, 'Aggressiveness': 95*20
            }])
            base_pred = model_clock.predict(dummy_input)[0]
            
            # YENİ OFFSET HESABI:
            # Gerçek Max (örn: 5449) - Modelin Ham Tahmini (örn: 5000) = Yeni Offset (449)
            # Bu offset artık "Silicon Quality" değerimizdir.
            new_offset = real_max_seen - base_pred
            
            # Güvenlik: Offset negatif veya çok düşük çıkarsa eskiyi koru
            if new_offset < c['Offset']: 
                final_offset = c['Offset'] # Eskisi kalsın
            else:
                final_offset = new_offset * 1.05 # %5 Tolerans payı ekle (Boost headroom)
            
            calibrated_cores.append({
                **c, 
                "Real_Max": real_max_seen,
                "Calibrated_Offset": final_offset
            })
            
            print(f"{c['Core']:<10} | {real_max_seen:.0f} MHz        | {c['Offset']:<12} | {final_offset:.0f} (REVİZE)")
            
    print("="*80)
    print("🚀 GÖREV: 5500 MHz SİMÜLASYONU (KALİBRE EDİLMİŞ)")
    print("   (Şimdi senin gördüğün 5449 MHz gerçeğiyle hesaplıyoruz)")
    print("="*80)

    for c in calibrated_cores:
        best_run = None
        max_reached = 0
        
        # 142W ve Limit CO ile deneme
        # Senin sistemin 95W'da bile 5449 görüyorsa, 142W'da daha rahat görmeli (ısı izin verirse)
        ppt_range = [95, 105, 115, 125, 135, 142]
        
        for ppt in ppt_range:
            input_data = pd.DataFrame([{
                'Set_PPT': ppt,
                'Set_CO': c['Limit_CO'], 
                'Set_Boost': 200,        
                'Fan': 1800,             
                'Temp_Lag1': 80,         
                'Clock_Lag1': 5000,
                'Aggressiveness': ppt * abs(c['Limit_CO'])
            }])
            
            pred_base = model_clock.predict(input_data)[0]
            pred_temp = model_temp.predict(input_data)[0]
            
            # KALİBRE OFFSETİ KULLAN
            final_clock = pred_base + c['Calibrated_Offset']
            
            # Sıcaklık cezası (Throttle): Eğer 95 dereceyi geçerse hız düşer
            if pred_temp > 95:
                final_clock -= (pred_temp - 95) * 15 # Derece başına 15 MHz kayıp
            
            if final_clock > max_reached: max_reached = final_clock
            
            if final_clock >= TARGET:
                best_run = {"PPT": ppt, "Temp": pred_temp, "Clock": final_clock}
                break 
        
        if best_run:
            status = "✅ MÜMKÜN"
            details = f"PPT: {best_run['PPT']}W | Tahmini Isı: {best_run['Temp']:.1f}°C"
            final_mhz = best_run['Clock']
        else:
            status = "⚠️ LİMİTTE"
            # Eğer 5450 üzerindeyse "Yaklaştı" diyelim
            if max_reached > 5450:
                status = "🤏 ÇOK YAKIN"
            details = f"Max: {max_reached:.0f} MHz"
            final_mhz = max_reached

        results.append({
            "Çekirdek": c['Core'],
            "Kalite": c['Rank'],
            "CO Limiti": c['Limit_CO'],
            "Tahmin (MHz)": f"{final_mhz:.0f}",
            "Durum": status,
            "Notlar": details
        })
        
    return pd.DataFrame(results)

# Çalıştır
if 'm_t' in locals() and 'm_c' in locals() and 'ranks' in locals() and 'X' in locals():
    # Full DF'i tekrar oluşturmak için load fonksiyonundan dönen veriye ihtiyacımız var
    # Hızlıca tekrar yükleyelim (hafızada yoksa diye)
    X_reload, y_t, y_c, ranks_reload = load_and_analyze_precision()
    # load_and_analyze_precision fonksiyonu full_df döndürmüyor, o yüzden içindeki df_list birleştirmesini simüle edelim
    # Pratik çözüm: load fonksiyonunu modifiye etmek yerine globalden çekmeyi deneyelim veya tekrar okuyalım.
    # En temizi: load fonksiyonunu çağırıp içindeki full_df'i döndürmesini sağlamaktı ama yukarıdaki hücrede onu yazmıştık.
    # Şimdilik yukarıdaki hücreyi tekrar çalıştırmana gerek yok, X verisi üzerinden gideceğiz.
    
    # HATA DÜZELTME: Calibrate fonksiyonu 'full_df' istiyor.
    # Bunu sağlamak için load fonksiyonunu küçük bir değişiklikle tekrar çağırıp full_df'i alalım.
    # (Aşağıdaki blok bunu otomatik yapar)
    
    print("Veri seti tekrar taranıyor (Kalibrasyon için)...")
    file_rules = [
        {"pattern": "Fans ve apu down", "PPT": 142, "CO": 0, "Boost": 0},
        {"pattern": "-30 CCD", "PPT": 95, "CO": -30, "Boost": 200},
        {"pattern": "65W-85W", "PPT": 65, "CO": -25, "Boost": 0},
        {"pattern": "95W-115W", "PPT": 95, "CO": -20, "Boost": 0}
    ]
    csv_files = glob.glob("*.csv") + glob.glob("*.CSV")
    df_list = []
    target_cols = {'Temp': 'CPU Tctl/Tdie °C', 'Clock': 'Çekirdek Etkili Saatler avg MHz', 'Power': 'CPU Tüm Güç Tüketimi W'}
    for f in csv_files:
        try:
            temp = pd.read_csv(f, low_memory=False, on_bad_lines='skip')
            temp.columns = [c.strip().replace('[', '').replace(']', '').replace('(', '').replace(')', '') for c in temp.columns]
            rename_map = {target_name: key for key, target_name in target_cols.items() if target_name in temp.columns}
            if len(rename_map) >= 3:
                temp = temp.rename(columns=rename_map)
                df_list.append(temp)
        except: pass
    full_df_calib = pd.concat(df_list, ignore_index=True)
    
    # Simülasyonu Çalıştır
    df_calibrated = calibrate_and_simulate_5500(m_t, m_c, ranks, full_df_calib)
    print(df_calibrated.to_markdown(index=False))
else:
    print("⚠️ Lütfen önce model eğitimi yapılan hücreyi çalıştırın.")